In [3]:
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import AveragePooling2D, Convolution2D, MaxPooling2D, ZeroPadding2D                               
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils
from keras.models import model_from_json
from keras.regularizers import l2
import keras
import h5py
import json



%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import sys
import time
import os, csv
import PIL
from PIL import Image
from numpy import genfromtxt
import pandas as pd

In [4]:
#funcitons used for script
def getImage(fileName):
    img = PIL.Image.open(fileName)                  #Opens as a color image
    img = img.convert('L')                          #Converts color image to greyscale
    img = img.resize((100,100), PIL.Image.ANTIALIAS)#resize to 100 by 100 for neural net
    return np.expand_dims(np.expand_dims(np.asarray(img),axis=0), axis=0)/255.0

In [7]:
#Load Networks

#segmentation network
with open('plateSegmentationNetwork.json', 'r') as f:
    segModelArchitecture = json.load(f)
segModel = model_from_json(segModelArchitecture)
#segModel.summary()

segModelWeights = 'plateSegmentationNetwork800.h5'
segModel.load_weights(segModelWeights)
#Compile the new model
segModel.compile(optimizer='rmsprop',loss='mse', metrics=['accuracy'])

#classifying network
with open('classifierBenArch.json', 'r') as f:
    classModelArchitecture = json.load(f)
classModel = model_from_json(classModelArchitecture)
#classModel.summary()

classModelWeights = 'classifierBenArch.h5'
classModel.load_weights(classModelWeights)

classModel.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

In [8]:
A = getImage('/Users/chris/Desktop/example4.jpg')
plt.imshow(A[0][0])
plt.show()
print segModel.predict(A)


IOError: [Errno 2] No such file or directory: '/Users/chris/Desktop/example4.jpg'